# Notebook 1: Análisis Exploratorio de Datos (EDA)

Este notebook utiliza los módulos de `src/` para realizar un análisis exploratorio de datos reproducible sobre el conjunto de datos de entrenamiento.

In [ ]:
import sys
import os
import pandas as pd

# Añadir la carpeta 'src' al path para poder importar los módulos locales
sys.path.append(os.path.abspath(os.path.join('..', 'src')))

from eda.eda_report import ejecutar_eda

## Configuración

Definimos las rutas a los datos y la columna objetivo.

In [ ]:
# Asumimos que los datos crudos están en data/raw/
RUTA_DATOS_ENTRENAMIENTO = '../data/raw/datos_entrenamiento_riesgo.csv'
COLUMNA_OBJETIVO = 'nivel_riesgo' # Adaptar si el nombre de la columna es diferente

## Ejecución del EDA

Llamamos a la función principal de nuestro script de EDA. Esta función se encargará de:
1. Cargar los datos.
2. Generar resúmenes estadísticos.
3. Crear y guardar visualizaciones (histogramas, boxplots, heatmaps, etc.).
4. Guardar todos los artefactos en el directorio `reports/`.

In [ ]:
if os.path.exists(RUTA_DATOS_ENTRENAMIENTO):
    ejecutar_eda(ruta_archivo=RUTA_DATOS_ENTRENAMIENTO, col_objetivo=COLUMNA_OBJETIVO)
else:
    print(f"Error: No se encontró el archivo de datos en '{RUTA_DATOS_ENTRENAMIENTO}'.")
    print("Por favor, asegúrate de que 'datos_entrenamiento_riesgo.csv' se encuentra en la carpeta 'data/raw'.")

## Resultados

Una vez ejecutado el script, puedes encontrar todos los reportes y figuras generados en el directorio `reports/` en la raíz del proyecto.